In [1]:
#Bag of words to explore the Notes data : Notes data,Lab data and Procedure data

Enter your name: Shreya
Hello Shreya


In [3]:
#%matplotlib inline

import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

notes= pd.read_csv('NOTEEVENTS.csv',error_bad_lines=False);
notes.shape
list(notes)

    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['ROW_ID',
 'SUBJECT_ID',
 'HADM_ID',
 'CHARTDATE',
 'CHARTTIME',
 'STORETIME',
 'CATEGORY',
 'DESCRIPTION',
 'CGID',
 'ISERROR',
 'TEXT']

In [4]:
print ('CHARTDATE')
print(notes['CHARTDATE'].iloc[0])
print ('CHARTTIME')
print(notes['CHARTTIME'].iloc[0])



CHARTDATE
2151-08-04
CHARTTIME
nan


In [5]:
#convert to dates- for consistency in format
notes.CHARTTIME= pd.to_datetime(notes.CHARTTIME, format ='%Y-%M-%D %H:%M:%S', errors ='coerce')
notes.STORETIME= pd.to_datetime(notes.STORETIME,format ='%Y-%M-%D %H:%M:%S', errors ='coerce')
print('CHARTTIME')
print(notes['CHARTTIME'].iloc[0])
print('STORETIME')
print(notes['STORETIME'].iloc[0])



CHARTTIME
nan
STORETIME
nan


In [6]:
print('CATEGORY')
print(notes['CATEGORY'].iloc[0])


CATEGORY
Discharge summary


In [7]:
lab=pd.read_csv('LABEVENTS.csv',error_bad_lines=False)
lab.shape
list(lab)


['ROW_ID',
 'SUBJECT_ID',
 'HADM_ID',
 'ITEMID',
 'CHARTTIME',
 'VALUE',
 'VALUENUM',
 'VALUEUOM',
 'FLAG']

In [8]:
df_notes_cate= notes.loc[notes['CATEGORY'].isin(['Discharge summary','Radiology'])]
print (df_notes_cate)




#assert df_notes_cate_clean.duplicated(['HADM_ID']).sum()==0,'Multiple discharge summaries per admission'

#notes['CATEGORY'] in ('Discharge Summary')
#notes['CATEGORY']

          ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE            CHARTTIME  \
0            174       22532  167853.0  2151-08-04                  NaN   
1            175       13702  107527.0  2118-06-14                  NaN   
2            176       13702  167118.0  2119-05-25                  NaN   
3            177       13702  196489.0  2124-08-18                  NaN   
4            178       26880  135453.0  2162-03-25                  NaN   
5            179       53181  170490.0  2172-03-08                  NaN   
6            180       20646  134727.0  2112-12-10                  NaN   
7            181       42130  114236.0  2150-03-01                  NaN   
8            182       56174  163469.0  2118-08-12                  NaN   
9            183       56174  189681.0  2118-12-09                  NaN   
10           184       28063  121936.0  2125-02-16                  NaN   
11           185        1136  139574.0  2192-05-23                  NaN   
12           223        5

In [14]:
proc=pd.read_csv('PROCEDURES_ICD.csv',error_bad_lines=False)
proc.shape
#list(proc)
#proc.columns
proc.groupby(['ICD9_CODE']).size()

df_proc_sep=proc.loc[proc['ICD9_CODE'].isin(['0202','3995','458'])]
print (df_proc_sep)
#df_proc_sep['ICD9_CODE'].unique() #check if we have everything



        ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM  ICD9_CODE
71        1015       10721   107493        2       3995
75        1019       25225   187093        1       3995
92        1036        3929   172648        2       3995
157       1101        7546   181171        2       3995
311        641        4113   146014        4       3995
374        704       17743   162994        2       3995
394        724        1701   191476        8       3995
401        731       25779   155168        1       3995
404        734       49034   159499        1       3995
428          7       87266   122187        2       3995
541        120       20409   102314        7       3995
580        159       23179   184235        2       3995
696        275       13413   117631        2       3995
725        304       23039   104767        1       3995
761        340       25708   197245        1       3995
762        341       25708   197245        2       3995
788        367       18738   147642        2    

In [17]:
df_lab=lab.head(1000)
df_proc_nsep=df_proc_sep.head(1000)



df_lab_proc=pd.merge(df_lab[['ROW_ID','SUBJECT_ID','HADM_ID','ITEMID','CHARTTIME','VALUE','VALUENUM','VALUEUOM','FLAG']],
                    df_proc_nsep[['ROW_ID','SUBJECT_ID','HADM_ID','SEQ_NUM','ICD9_CODE']],
                    on= ['SUBJECT_ID','HADM_ID'],
                    how='outer')
df_lab_proc.columns

Index(['ROW_ID_x', 'SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUE',
       'VALUENUM', 'VALUEUOM', 'FLAG', 'ROW_ID_y', 'SEQ_NUM', 'ICD9_CODE'],
      dtype='object')

In [18]:
df_notes_cate_ten=df_notes_cate.head(1000)
print (df_notes_cate_ten)

#df_lab=lab.head(1000)
#print (df_lab)

#df_proc=proc.head(1000)

df_lab_notes_proc= pd.merge(df_lab_proc[['ROW_ID_x','SUBJECT_ID','HADM_ID','ITEMID','CHARTTIME','VALUE','VALUENUM','VALUEUOM','FLAG','ROW_ID_y','SEQ_NUM','ICD9_CODE']],    
                 df_notes_cate_ten[['ROW_ID','SUBJECT_ID','HADM_ID','CHARTDATE','CHARTTIME','STORETIME','CATEGORY','DESCRIPTION','CGID','ISERROR','TEXT']],
                  on= ['SUBJECT_ID','HADM_ID'],
                    how ='outer')

print(df_lab_notes_proc)

     ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE CHARTTIME STORETIME  \
0       174       22532  167853.0  2151-08-04       NaN       NaN   
1       175       13702  107527.0  2118-06-14       NaN       NaN   
2       176       13702  167118.0  2119-05-25       NaN       NaN   
3       177       13702  196489.0  2124-08-18       NaN       NaN   
4       178       26880  135453.0  2162-03-25       NaN       NaN   
5       179       53181  170490.0  2172-03-08       NaN       NaN   
6       180       20646  134727.0  2112-12-10       NaN       NaN   
7       181       42130  114236.0  2150-03-01       NaN       NaN   
8       182       56174  163469.0  2118-08-12       NaN       NaN   
9       183       56174  189681.0  2118-12-09       NaN       NaN   
10      184       28063  121936.0  2125-02-16       NaN       NaN   
11      185        1136  139574.0  2192-05-23       NaN       NaN   
12      223        5350  169684.0  2143-04-30       NaN       NaN   
13      224        5350  169684.0 

In [ ]:
#Prepare a column in the dataframe that would filter our that seperate data just for systolic blood pressure <90mmHg  
#and fluid replacement.

#df_lab_notes['SYSTOLIC_BLOOD']= (df_lab_notes.VALUENUM

In [35]:
#shuffle the sample to make a training set,validation set and test set for the predictive model- 
df_lab_notes_proc=df_lab_notes_proc.sample( n= len(df_lab_notes_proc),random_state= 42)
df_lab_notes_proc= df_lab_notes_proc.reset_index(drop=True)

df_test_valid=df_lab_notes_proc.sample(frac=0.30,random_state=42) #save 30% of the data as valid and test

df_test= df_test_valid.sample(frac= 0.5,random_state=42)
df_valid= df_test_valid.drop(df_test.index)

df_train_all=df_lab_notes_proc.drop(df_test_valid.index) #use the rest of the data as training data

print(df_train_all)

#check if we have sampled the data well. Check for prevalence.
print ('Test prevalence of sepsis related case(n= %d):' %len(df_test),df_test['ICD9_CODE'].isin(['3995','0202','458']).sum()/len(df_test))
print('Valid prevalence (n= %d):' %len(df_valid),df_valid['ICD9_CODE'].isin(['3995','0202','458']).sum()/len(df_valid))
print ('Train all prevalence (n= %d):' %len(df_train_all),df_train_all['ICD9_CODE'].isin(['3995','0202','458']).sum()/len(df_train_all))
print('all sample (n= %d)'%len(df_lab_notes_proc))



      ROW_ID_x  SUBJECT_ID   HADM_ID   ITEMID          CHARTTIME_x  \
1          NaN       16394  128478.0      NaN                  NaN   
2          NaN       26274  126494.0      NaN                  NaN   
3       1133.0           3  145834.0  50812.0  2101-10-23 10:14:00   
4          NaN        1544  119646.0      NaN                  NaN   
5          NaN       16790  105717.0      NaN                  NaN   
6        394.0           3       NaN  50800.0  2101-10-15 12:42:00   
8       1004.0           3  145834.0  50816.0  2101-10-22 11:21:00   
9          NaN        9148  129601.0      NaN                  NaN   
10         NaN       11827  144355.0      NaN                  NaN   
11         NaN        1106  188969.0      NaN                  NaN   
13       619.0           3  145834.0  50812.0  2101-10-21 00:42:00   
14       511.0           3  145834.0  50800.0  2101-10-20 19:14:00   
15       949.0           3  145834.0  51222.0  2101-10-22 04:00:00   
16       363.0      

In [22]:
#Bag of Words

#Preprocessing the data
def preprocess_text(df):
    df.TEXT=df.TEXT.fillna('')
    df.TEXT=df.TEXT.str.replace('\n','')
    df.TEXT=df.TEXT.str.replace('\r','')
    return df






In [46]:
df_train_all= preprocess_text(df_train_all)
df_valid=preprocess_text(df_valid)
df_test=preprocess_text(df_test)

In [47]:
#Build a tokenizer - this could be of your choice;you can replace punctations with spaces,remove numbers with spaces and lowercase all words.

import nltk
nltk.download('punkt') #didnt have this in my library yet. So downloaded here.
from nltk import word_tokenize
word_tokenize('This is a sample to tokenize. 11/21/2018 sentence needs space**')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shreyavarghese/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['This',
 'is',
 'a',
 'sample',
 'to',
 'tokenize',
 '.',
 '11/21/2018',
 'sentence',
 'needs',
 'space**']

In [48]:
import string
print (string.punctuation)


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [49]:
def tokenizer_better(text):
#will remove spaces , numbers and punctuations-will just tokenize test
    punc_list=string.punctuation+'0123456789'
    t= str.maketrans(dict.fromkeys(punc_list,""))
    text = text.lower().translate(t)
    tokens =word_tokenize(text)
    return tokens


In [50]:
tokenizer_better('This is a sample to test tokenizer.11/21/2018 sentence needs space**')

['this',
 'is',
 'a',
 'sample',
 'to',
 'test',
 'tokenizer',
 'sentence',
 'needs',
 'space']

In [51]:
sample_text= ['This research is exploratory analysis', 'Bag of words and its strengths','Can we explore sepsis data']

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
vect= CountVectorizer (tokenizer=tokenizer_better)
vect.fit (sample_text)

X=vect.transform(sample_text)

In [53]:
X

<3x16 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [54]:
X.toarray()

array([[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0]])

In [55]:
vect.get_feature_names()


['analysis',
 'and',
 'bag',
 'can',
 'data',
 'exploratory',
 'explore',
 'is',
 'its',
 'of',
 'research',
 'sepsis',
 'strengths',
 'this',
 'we',
 'words']

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
vect= CountVectorizer(max_features=3000, tokenizer =tokenizer_better)

vect.fit(df_train_all.TEXT.values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=3000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenizer_better at 0x119384510>,
        vocabulary=None)

In [68]:
import matplotlib.pyplot as plt

In [57]:
#Fitting the vectorized data to fit and converting it into an array: Step is important to fit the data in the bag of words 
#and within the Pandas dataframe.
doc_matrix= vect.transform(df_train_all.TEXT)
doc_tf=np.sum(doc_matrix,axis=0)
doc=np.squeeze(np.asarray(doc_tf))



term_freq_df=pd.DataFrame([doc],columns=vect.get_feature_names()).transpose()
print (term_freq_df)


#d = pd.Series(term_freq_df,index = term_freq_df.index).sort_values(ascending=False)
#print (d)

#ax = term_freq_df[:50].plot(kind='bar', figsize=(10,6), width=.8, fontsize=14, rot=90,color = 'b')
#ax.title.set_size(18)
#plt.ylabel('count')
#plt.show()
#ax = term_freq_df[50:100].plot(kind='bar', figsize=(10,6), width=.8, fontsize=14, rot=90,color = 'b')
#ax.title.set_size(18)
#plt.ylabel('count')
#plt.show()


                   0
a              10892
abd               92
abdomen          344
abdominal        574
abdominalpain     28
abg               84
ablation          34
able             313
abnormal         100
abnormalities     92
abnormality       78
about            233
above            288
abscess          110
absence           48
absent            37
abuse            117
abx               47
ac                70
access            88
accessory         45
accident          32
according         45
ace               46
acei              48
acetaminophen    195
acetate           44
acid             223
acidosis          82
acquired          31
...              ...
worse             75
worsened          53
worsening        220
would            308
wound            263
wounds            50
x               1165
xl                86
xray             302
xrt               31
xs               136
year             418
yearold          214
years            334
yeast             87
yeastnoneepi 

In [58]:
my_stop_words = ['the','and','to','of','was','with','a','on','in','for','name',
                 'is','patient','s','he','at','as','or','one','she','his','her','am',
                 'were','you','pt','pm','by','be','had','your','this','date',
                'from','there','an','that','p','are','have','has','h','but','o',
                'namepattern','which','every','also']

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
vect= CountVectorizer(max_features= 500,
                     tokenizer=tokenizer_better,
                     stop_words=my_stop_words)

vect.fit(df_train_all.TEXT.values)


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=500, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['the', 'and', 'to', 'of', 'was', 'with', 'a', 'on', 'in', 'for', 'name', 'is', 'patient', 's', 'he', 'at', 'as', 'or', 'one', 'she', 'his', 'her', 'am', 'were', 'you', 'pt', 'pm', 'by', 'be', 'had', 'your', 'this', 'date', 'from', 'there', 'an', 'that', 'p', 'are', 'have', 'has', 'h', 'but', 'o', 'namepattern', 'which', 'every', 'also'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenizer_better at 0x119384510>,
        vocabulary=None)

In [60]:
A_train_tf=vect.transform(df_train_all.TEXT.values)
A_valid_tf=vect.transform(df_valid.TEXT.values)